## Sentinel-2 vegetation mask based on NDVI and BSI

### Service Definition

In [ ]:
service = dict([('title', 'Sentinel-2 vegetation mask based on NDVI and BSI'),
                ('abstract', 'Sentinel-2 vegetation mask based on NDVI and BSI'),
                ('id', 'ewf-s2-vegetation-mask')])

**Number of classes**

Number of modes, which will be used to generate class membership

In [ ]:
n_classes = dict([('id', 'n_classes'),
                  ('value', '1=-1#0,2=0#0.1,3=0.1#0.2,4=0.2#1'),
                  ('title', 'Classes and ranges for NDVI'),
                  ('abstract', 'Classes and ranges for NDVI (class_1=min|max,class_2=min|max)'),
                  ('maxOccurs', '20')])

In [ ]:
b_classes = dict([('id', 'b_classes'),
                  ('value', '1=-1#-0.2,2=-0.2#0.1,3=0.1#0.2,4=0.2#1'),
                  ('title', 'Classes and ranges for BSI'),
                  ('abstract', 'Classes and ranges for BSI (class_1=min|max,class_2=min|max)'),
                  ('maxOccurs', '20')])

In [ ]:
ndvi_threshold = dict([('id', 'ndvi_threshold'),
                   ('value', '0.3'),
                   ('title', 'NDVI threshold for the mask expression'),
                   ('abstract', 'NDVI threshold for the mask expression'),
                   ('maxOccurs', '1')])

In [ ]:
bsi_threshold = dict([('id', 'bsi_threshold'),
                   ('value', '0'),
                   ('title', 'BSI threshold'),
                   ('abstract', 'BSI threshold'),
                   ('maxOccurs', '1')])

In [ ]:
aoi = dict([('id', 'aoi'),
              ('title', 'Area of interest'),
              ('abstract', 'Area of interest'),
              ('value', '-70.5659,-13.0922,-69.1411,-12.4567')])

In [ ]:
username = dict([('id', '_T2Username'),
              ('title', 'Ellip username'),
              ('abstract', 'Ellip username'),
              ('value', '')])

In [ ]:
api_key = dict([('id', '_T2ApiKey'),
              ('title', 'Ellip API key for data pipeline'),
              ('abstract', 'Ellip API key for data pipeline'),
              ('value', '')])

### <a name="runtime">Runtime parameter definition

**Input reference**

This is the Sentinel-2 catalogue entry URLs

In [ ]:
input_references = ['https://catalog.terradue.com/ard-s2-boa-reflectances/search?uid=6276B1444A67F9C3C38F330D8A09B3462FF1836A',
                    'https://catalog.terradue.com/ard-s2-boa-reflectances/search?uid=D2CC3A1F5F9E61D22CDF7F7513A1A35E6AA76A0D',
                    'https://catalog.terradue.com/ard-s2-boa-reflectances/search?uid=AB3807E0B5158644C1052951F109C717FC62D289',
                    'https://catalog.terradue.com/ard-s2-boa-reflectances/search?uid=39BE1E4541AFD99D538E1FE32694E84B3E58FD7E']

### Workflow

#### Import the packages required for processing the data

In [ ]:
import os
import sys

sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())
from helpers import *
%load_ext autoreload
%autoreload 2


sys.path.append('/opt/OTB/lib/python')
sys.path.append('/opt/OTB/lib/libfftw3.so.3')
os.environ['OTB_APPLICATION_PATH'] = '/opt/OTB/lib/otb/applications'
os.environ['LD_LIBRARY_PATH'] = '/opt/OTB/lib'
os.environ['ITK_AUTOLOAD_PATH'] = '/opt/OTB/lib/otb/applications'
os.environ['GDAL_DATA'] = '/opt/anaconda/share/gdal/'
import otbApplication

In [ ]:
product_metadata = get_product_metadata(input_references, 
                                        username['value'],
                                        api_key['value'])

In [ ]:
product_metadata

In [ ]:
min_date = min(product_metadata['startdate'])
max_date = max(product_metadata['enddate'])

#### Area of interest

In [ ]:
x_min, y_min, x_max, y_max = [float(c) for c in aoi['value'].split(',')]

In [ ]:
x_min, y_min, x_max, y_max

#### Input vsi URLs

Use the Sentinel-2 ARD using GDAL Virtual File System

In [ ]:
vsi_list = [get_vsi_url(input_reference, 
                        username['value'],
                        api_key['value']) for input_reference in input_references]

Create a GDAL virtual dataset with the VSI URLs

In [ ]:
vrt_options = gdal.BuildVRTOptions()

vrt = 'my.vrt'

ds_vrt = gdal.BuildVRT(vrt, vsi_list, options=vrt_options)
ds_vrt.FlushCache()

In [ ]:
src_ds = gdal.Open(get_vsi_url(input_references[0], 
                               username['value'],
                               api_key['value']))

ds_vrt = gdal.Open(vrt,  gdal.OF_UPDATE)

for band in range(ds_vrt.RasterCount):
    
    band += 1
    print band, src_ds.GetRasterBand(band).GetDescription()
    
    src_band = ds_vrt.GetRasterBand(band)
    src_band.SetDescription(src_ds.GetRasterBand(band).GetDescription())  
    
ds_vrt.FlushCache()

Create a new VRT with the Area of Interest (easier to do with gdal.Translate)

In [ ]:
clipped_vrt = 'clipped.vrt'

gdal.Translate(clipped_vrt,
               vrt,
               projWin=[x_min, y_max, x_max, y_min],
               projWinSRS='EPSG:4326',
               format='VRT')

#### Band expressions

In [ ]:
band_2 = 'im1b2'
band_4 = 'im1b4'
band_8 = 'im1b8'
band_11 = 'im1b11'
scl = 'im1b13'

In [ ]:
ndvi_expression = '({1}-{0})/({1}+{0})'.format(band_4, band_8)
bsi_expression = '(({3}/{4}+{1}/{4})-({2}/{4}+{0}/{4}))/(({3}/{4}+{1}/{4})+({2}/{4}+{0}/{4}))'.format(band_2, band_4, band_8, band_11, '10000')
invalid_expression = '{0} == 0 || {0} == 1 || {0} == 3 || {0} == 8 || {0} == 9'.format(scl) 


# if ndvi  >= 0.3 and bsi <= 0 then IS vegetation, otherwise no vegetation
mask_expression = '{0} ? 128 : {1} >= {2} && {3} <= {4} ? 1 : 0'.format(invalid_expression,
                                                                        ndvi_expression, 
                                                                        ndvi_threshold['value'],
                                                                        bsi_expression,
                                                                        bsi_threshold['value'])



In [ ]:
ndvi_expression, bsi_expression, invalid_expression, mask_expression


**Vegetation index classes**

In [ ]:
ndvi_classes = dict((int(k.strip()), v.strip().replace('#', ',')) for k,v in  
                             (item.split('=') for item in n_classes['value'].split(',')))


bsi_classes = dict((int(k.strip()), v.strip().replace('#', ',')) for k,v in  
                             (item.split('=') for item in b_classes['value'].split(',')))

In [ ]:
ndvi_classes, bsi_classes

** All expressions **

In [ ]:
expressions = []

invalid_data = 128

for index, vi_class in enumerate([ndvi_classes, bsi_classes]):
    
    f = '{} ? {}'.format(invalid_expression, invalid_data)

    for _class in vi_class.keys():

        if index == 0: vi_expression = ndvi_expression
        if index == 1: vi_expression = bsi_expression
            
        expression = '{0} >= {1} && {0} < {2} ? {3}'.format(vi_expression,
                                                                vi_class[_class].split(',')[0],
                                                                vi_class[_class].split(',')[1],
                                                                _class)
                   

        f = '{} : {} '.format(f, expression)

    f = '{}: {}'.format(f, 0)
    
    expressions.append(f)
    
expressions.append('{} ? {} : 0'.format(invalid_expression, invalid_data))

expressions.append(mask_expression)

In [ ]:
expressions

Provide a meaningful output name for the result

In [ ]:
date_format = '%Y%m%dT%H%m%S'

output_name = 'VEGETATION-MASK-{0}-{1}'.format(min_date.strftime(date_format), 
                                               max_date.strftime(date_format))

Apply the Orfeo Toolbox BandMathX operator

In [ ]:
BandMathX = otbApplication.Registry.CreateApplication('BandMathX')

BandMathX.SetParameterStringList('il', [clipped_vrt])
BandMathX.SetParameterString('out', 'temp_{0}.tif'.format(output_name))
BandMathX.SetParameterString('exp', ';'.join(expressions))
BandMathX.SetParameterOutputImagePixelType('out', otbApplication.ImagePixelType_uint8)

BandMathX.ExecuteAndWriteOutput()

In [ ]:
band_names = ['ndvi_class',
             'bsi_class',
             'cloud_mask',
             'vegetation_mask']

metadata = dict()
metadata['B02'] = 'im1b2'
metadata['B04'] = 'im1b4'
metadata['B08'] = 'im1b8'
metadata['B11'] = 'im1b11'
metadata['SCL'] = 'im1b13'


ds_temp = gdal.Open('temp_{0}.tif'.format(output_name),  gdal.OF_UPDATE)

for band_index in range(ds_temp.RasterCount):
    

    metadata['BAND_EXPRESSION'] = '({})'.format(expressions[band_index])
    
    src_band = ds_temp.GetRasterBand(band_index+1)
    src_band.SetMetadata(metadata)
    src_band.SetDescription(band_names[band_index])  
    
ds_temp.FlushCache()


Transform it to a Cloud Optimized GeoTIFF

In [ ]:
cog('temp_{0}.tif'.format(output_name),
    '{0}.tif'.format(output_name))

#### Create an RGB quicklook

In [ ]:
mask_expression_r = '{0} ? 255 : {1} >= {2} && {3} <= {4} ? 0 : 0'.format(invalid_expression,
                                                                         ndvi_expression, 
                                                                         ndvi_threshold['value'],
                                                                         bsi_expression,
                                                                         bsi_threshold['value'])

mask_expression_g = '{0} ? 255 : {1} >= {2} && {3} <= {4} ? 255 : 0'.format(invalid_expression,
                                                                          ndvi_expression, 
                                                                          ndvi_threshold['value'],
                                                                          bsi_expression,
                                                                          bsi_threshold['value'])


mask_expression_b = '{0} ? 255 : {1} >= {2} && {3} <= {4} ? 0 : 255'.format(invalid_expression,
                                                                         ndvi_expression, 
                                                                         ndvi_threshold['value'],
                                                                         bsi_expression,
                                                                         bsi_threshold['value'])

In [ ]:
BandMathX = otbApplication.Registry.CreateApplication('BandMathX')

BandMathX.SetParameterStringList('il', [clipped_vrt])
BandMathX.SetParameterString('out', 'temp_{0}.rgb.tif'.format(output_name))
BandMathX.SetParameterString('exp', ';'.join([mask_expression_r, mask_expression_g, mask_expression_b]))
BandMathX.SetParameterOutputImagePixelType('out', otbApplication.ImagePixelType_uint8)

BandMathX.ExecuteAndWriteOutput()

In [ ]:
cog('temp_{0}.rgb.tif'.format(output_name),
    '{0}.rgb.tif'.format(output_name))

#### Metadata

Create the results metadata

In [ ]:
for properties_file in ['result', output_name]:

    date_format = '%Y-%m-%dT%H:%m:%SZ'
    
    if properties_file == 'result':
        
        title = 'Reproducibility notebook used for generating {0}'.format(output_name)
   
    else: 
      
        title = 'Vegetation mask from {0} to {1}'.format(min_date.strftime(date_format),
                                                         max_date.strftime(date_format))
        
    with open(properties_file + '.properties', 'wb') as file:
        
        file.write('title={0}\n'.format(title))
        
        file.write('date={0}/{1}\n'.format(min_date.strftime(date_format),
                                           max_date.strftime(date_format)))
        
        file.write('geometry={0}'.format(get_image_wkt(output_name + '.tif')))

#### Clean-up

In [ ]:
os.remove(clipped_vrt)
os.remove(vrt)

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.